In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'apple-disease-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2068739%2F3433042%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240520%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240520T053237Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D2130868ada93cc9abfe0e665e99863a98a46fc678b2b67135ff484eeae143bbea9c5bb0b85a03a4e79877535e24411498e6909550c83f9e466fcd2e156cffb6317080f4b26c3fc6b6a200fbe952c42babfe1d6861d35f88db7913184d04741efbdfcb4e1c615683265a84348a8d5e00c075dae13e85e7cb276b1dfaffef54d0b241caee808b191a83e6a6105b8170b0f0d92e2d26e31e83e151f0d8ba096f450903108fe7724b2720f7edea3f890650a4f7a13be8fde6445f52465d4ba55504642aaea0b54987f3fbfb7577a68f62bc3ed53f26138a3946f1afb62f7a44db79eb328f6050ca3b4fb5274083e047d24222117c4f03cd47822c89154c04fe9ad25,d-kap:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2464177%2F4176149%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240520%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240520T053237Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D39c4c91aac7f38789eea67d33f7d132d077a421e5d78b8f1c0618f70e80d446150b2f70190cfad3843fd399a519b96e985b8de92cfb547e9773772305a0e3ce83d9ec8e80f3c5979f7106566f11b79f46a77b5289193ffd1ae71d2e7878976e0c66501cf8fc35e3c5badb72070b011a6909236b2ad77454c5174bd86e54546419598c63950f77bbe117d346c965cb8908abb6c047eb4d31fa1b870c0422e7e42309186caf215fbb596572d2d61d268ef6eee805350446eb4eee5b63c670171e1ef3f803253593b7c4688b2bb7dcda368644fef32d50481e824e4371c8dcd19263e13fb834f85b1e336e467263977ae6f882f2f886b0cee17625e11705bc51984'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import shutil
import os

# Set the source directory
src_dir = "/kaggle/input/apple-disease-dataset/datasets/train"

# Set the destination directory
dst_dir = "/kaggle/working/apple-disease-dataset/datasets/train"

# Remove the destination directory if it already exists
if os.path.exists(dst_dir):
    shutil.rmtree(dst_dir)

# Copy the contents of the source directory to the destination directory
shutil.copytree(src_dir, dst_dir)

In [ ]:
import shutil
import os

# Set the source directory
src_dir = "/kaggle/input/apple-disease-dataset/datasets/test"

# Set the destination directory
dst_dir = "/kaggle/working/apple-disease-dataset/datasets/test"

# Remove the destination directory if it already exists
if os.path.exists(dst_dir):
    shutil.rmtree(dst_dir)

# Copy the contents of the source directory to the destination directory
shutil.copytree(src_dir, dst_dir)

In [ ]:
import os
import random
import shutil

# Define paths to source directory and two destination directories
src_dir = r'/kaggle/input/d-kap/APPLE_DISEASE_DATASET/APPLE ROT LEAVES'
dst_dir_1 = '/kaggle/working/apple-disease-dataset/datasets/train/black_rot'
dst_dir_2 = '/kaggle/working/apple-disease-dataset/datasets/test/black_rot'

# Create the destination directories if they don't already exist
if not os.path.exists(dst_dir_1):
    os.makedirs(dst_dir_1)
if not os.path.exists(dst_dir_2):
    os.makedirs(dst_dir_2)

# Get a list of all files in the source directory
files = os.listdir(src_dir)

# Calculate the number of files to copy to each destination directory (60% and 40%)
num_files_to_copy_1 = int(len(files) * 0.6)
num_files_to_copy_2 = len(files) - num_files_to_copy_1

# Randomly select the files to copy to each destination directory
files_to_copy_1 = random.sample(files, num_files_to_copy_1)
files_to_copy_2 = list(set(files) - set(files_to_copy_1))

# Copy the selected files to each destination directory
for file_name in files_to_copy_1:
    src_file_path = os.path.join(src_dir, file_name)
    dst_file_path = os.path.join(dst_dir_1, file_name)
    shutil.copy(src_file_path, dst_file_path)

for file_name in files_to_copy_2:
    src_file_path = os.path.join(src_dir, file_name)
    dst_file_path = os.path.join(dst_dir_2, file_name)
    shutil.copy(src_file_path, dst_file_path)

In [ ]:
import os
import random
import shutil

# Define paths to source directory and two destination directories
src_dir = r'/kaggle/input/d-kap/APPLE_DISEASE_DATASET/HEALTHY LEAVES'
dst_dir_1 = '/kaggle/working/apple-disease-dataset/datasets/train/healthy'
dst_dir_2 = '/kaggle/working/apple-disease-dataset/datasets/test/healthy'

# Create the destination directories if they don't already exist
if not os.path.exists(dst_dir_1):
    os.makedirs(dst_dir_1)
if not os.path.exists(dst_dir_2):
    os.makedirs(dst_dir_2)

# Get a list of all files in the source directory
files = os.listdir(src_dir)

# Calculate the number of files to copy to each destination directory (60% and 40%)
num_files_to_copy_1 = int(len(files) * 0.6)
num_files_to_copy_2 = len(files) - num_files_to_copy_1

# Randomly select the files to copy to each destination directory
files_to_copy_1 = random.sample(files, num_files_to_copy_1)
files_to_copy_2 = list(set(files) - set(files_to_copy_1))

# Copy the selected files to each destination directory
for file_name in files_to_copy_1:
    src_file_path = os.path.join(src_dir, file_name)
    dst_file_path = os.path.join(dst_dir_1, file_name)
    shutil.copy(src_file_path, dst_file_path)

for file_name in files_to_copy_2:
    src_file_path = os.path.join(src_dir, file_name)
    dst_file_path = os.path.join(dst_dir_2, file_name)
    shutil.copy(src_file_path, dst_file_path)

In [ ]:
import os
import random
import shutil

# Define paths to source directory and two destination directories
src_dir = r'/kaggle/input/d-kap/APPLE_DISEASE_DATASET/LEAF BLOTCH'
dst_dir_1 = '/kaggle/working/apple-disease-dataset/datasets/train/cedar_apple_rust'
dst_dir_2 = '/kaggle/working/apple-disease-dataset/datasets/test/cedar_apple_rust'

# Create the destination directories if they don't already exist
if not os.path.exists(dst_dir_1):
    os.makedirs(dst_dir_1)
if not os.path.exists(dst_dir_2):
    os.makedirs(dst_dir_2)

# Get a list of all files in the source directory
files = os.listdir(src_dir)

# Calculate the number of files to copy to each destination directory (60% and 40%)
num_files_to_copy_1 = int(len(files) * 0.6)
num_files_to_copy_2 = len(files) - num_files_to_copy_1

# Randomly select the files to copy to each destination directory
files_to_copy_1 = random.sample(files, num_files_to_copy_1)
files_to_copy_2 = list(set(files) - set(files_to_copy_1))

# Copy the selected files to each destination directory
for file_name in files_to_copy_1:
    src_file_path = os.path.join(src_dir, file_name)
    dst_file_path = os.path.join(dst_dir_1, file_name)
    shutil.copy(src_file_path, dst_file_path)

for file_name in files_to_copy_2:
    src_file_path = os.path.join(src_dir, file_name)
    dst_file_path = os.path.join(dst_dir_2, file_name)
    shutil.copy(src_file_path, dst_file_path)

In [ ]:
import os
import random
import shutil

# Define paths to source directory and two destination directories
src_dir = r'/kaggle/input/d-kap/APPLE_DISEASE_DATASET/SCAB LEAVES'
dst_dir_1 = '/kaggle/working/apple-disease-dataset/datasets/train/apple_scab'
dst_dir_2 = '/kaggle/working/apple-disease-dataset/datasets/test/apple_scab'

# Create the destination directories if they don't already exist
if not os.path.exists(dst_dir_1):
    os.makedirs(dst_dir_1)
if not os.path.exists(dst_dir_2):
    os.makedirs(dst_dir_2)

# Get a list of all files in the source directory
files = os.listdir(src_dir)

# Calculate the number of files to copy to each destination directory (60% and 40%)
num_files_to_copy_1 = int(len(files) * 0.6)
num_files_to_copy_2 = len(files) - num_files_to_copy_1

# Randomly select the files to copy to each destination directory
files_to_copy_1 = random.sample(files, num_files_to_copy_1)
files_to_copy_2 = list(set(files) - set(files_to_copy_1))

# Copy the selected files to each destination directory
for file_name in files_to_copy_1:
    src_file_path = os.path.join(src_dir, file_name)
    dst_file_path = os.path.join(dst_dir_1, file_name)
    shutil.copy(src_file_path, dst_file_path)

for file_name in files_to_copy_2:
    src_file_path = os.path.join(src_dir, file_name)
    dst_file_path = os.path.join(dst_dir_2, file_name)
    shutil.copy(src_file_path, dst_file_path)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

# Load the dataset
train_data = datagen.flow_from_directory('/kaggle/working/apple-disease-dataset/datasets/train', target_size=(224,224), batch_size=32, class_mode='categorical')
val_data = datagen.flow_from_directory('/kaggle/working/apple-disease-dataset/datasets/test', target_size=(224,224), batch_size=32, class_mode='categorical')

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2) # set validation split to 0.2

train_data = datagen.flow_from_directory('/kaggle/working/apple-disease-dataset/datasets/train',
                                         target_size=(224,224),
                                         batch_size=32,
                                         class_mode='categorical',
                                         subset='training') # set subset to 'training' for train data

val_data = datagen.flow_from_directory('/kaggle/working/apple-disease-dataset/datasets/train',
                                       target_size=(224,224),
                                       batch_size=32,
                                       class_mode='categorical',
                                       subset='validation') # set subset to 'validation' for validation data

test_data = datagen.flow_from_directory('/kaggle/working/apple-disease-dataset/datasets/test',
                                         target_size=(224,224),
                                         batch_size=32,
                                         class_mode='categorical')

In [ ]:
# Get x_train and y_train
x_train, y_train = next(train_data)

x_val, y_val = next(val_data)
# Get x_test and y_test
x_test, y_test = next(test_data)

In [ ]:
import tensorflow as tf

model_1=tf.keras.models.load_model('/kaggle/input/models/best_model_RESNET50_actual.h5')
model_2=tf.keras.models.load_model('/kaggle/input/models/final_model_VGG19.h5')
model_3=tf.keras.models.load_model('/kaggle/input/models/final_model_densenet201.h5')
model_4=tf.keras.models.load_model('/kaggle/input/models/final_model_efficientnet.h5')
model_5=tf.keras.models.load_model('/kaggle/input/models/final_model_mobilenetv2.h5')

In [ ]:
train_loss, train_acc = model_1.evaluate(train_data)
val_loss, val_acc = model_1.evaluate(val_data)

print(f"Final training accuracy: {train_acc:.4f}")
print(f"Final validation accuracy: {val_acc:.4f}")

In [ ]:
model_1

In [ ]:
from itertools import combinations

# create a list of all possible combinations of base models with length greater than equal 2
base_models = [model_1, model_2, model_3, model_4, model_5]
base_model_combinations = []
for i in range(2, len(base_models) + 1):
    base_model_combinations += combinations(base_models, i)


In [ ]:
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam



In [ ]:
import numpy as np
import pandas as pd

# create a dictionary to store the predictions of base models on the validation set
base_model_predictions = {}

# loop over all base model combinations
for combination in base_model_combinations:
    # create a list to store the predictions of the current base model combination
    combination_predictions = []

    # loop over all base models in the current combination
    for base_model in combination:
        # get the predictions of the current base model on the validation set
        predictions = base_model.predict(x_train)
        combination_predictions.append(predictions)

    # concatenate the predictions of all base models in the current combination
    combination_predictions = np.concatenate(combination_predictions, axis=1)

    # store the predictions of the current base model combination in the dictionary
    base_model_predictions[combination] = combination_predictions


In [ ]:
list(base_model_predictions.values())[0].shape

In [ ]:
y_val.shape

In [ ]:
np.tile(x_val,3).shape

In [ ]:


combination_accuracies = []
# loop over all base model combinations
for combination, predictions in base_model_predictions.items():
    # define the meta model architecture
    meta_model = Sequential([Dense(152, activation='relu', input_shape=(len(combination)*4,)),
                             Dropout(0.5),
                             Dense(64, activation='relu'),
                             Dropout(0.3),
                         Dense(4, activation='softmax')])
    meta_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # train the meta model on the predictions of the current base model combination
    meta_model.fit(predictions, y_train, epochs=10, batch_size=32, verbose=0)

    # get the predictions of the current base model combination on the test set
    test_predictions = []

    train_predictions=[]
    for base_model in combination:
        predictions_train = base_model.predict(x_train)
        train_predictions.append(predictions_train)
    train_predictions = np.concatenate(train_predictions, axis=1)

    for base_model in combination:
        predictions_test = base_model.predict(x_test)
        test_predictions.append(predictions_test)
    test_predictions = np.concatenate(test_predictions, axis=1)

    # evaluate the meta model on the test set
    _, test_accuracy = meta_model.evaluate(test_predictions, y_test, verbose=0)
    _, train_accuracy = meta_model.evaluate(train_predictions, y_train, verbose=0)
    # print the accuracy of the current base model combination
    print(f'Accuracy for {len(combination)} base models_train: {train_accuracy},base_model_test:{test_accuracy}')
    #print(f'Accuracy for {len(combination)} ,base_model_test:{test_accuracy}')

    # add the accuracy of the current base model combination to the list
    combination_accuracies.append((combination,train_accuracy,test_accuracy))


In [ ]:
sorted(combination_accuracies,key=lambda x: (x[2], x[1],5-len(x[0])),reverse=True)[0:10]
#[1,4,5]
#[2,3,4]
#[2,3,5]

In [ ]:
# loop through all layers in the model
for layer in model_1.layers:
    # set the new name for the layer
    layer._name = 'model_1_' + layer.name
for layer in model_2.layers:
    # set the new name for the layer
    layer._name = 'model_2_' + layer.name
for layer in model_3.layers:
    # set the new name for the layer
    layer._name = 'model_3_' + layer.name
for layer in model_4.layers:
    # set the new name for the layer
    layer._name = 'model_4_' + layer.name
for layer in model_5.layers:
    # set the new name for the layer
    layer._name = 'model_5_' + layer.name



In [ ]:
#[1,4,5]
from keras.regularizers import l2
from keras.models import Model
from keras.layers import concatenate, Dense, Dropout

# assume you have three models named model1, model2, model3
# and a meta-model named dense_meta_model
model_1.trainable = False
model_4.trainable=False
model_5.trainable = False
# get the output layers of individual models
output1 = model_1.output
output2 = model_4.output
output3 = model_5.output

# concatenate the outputs of the individual models
merged = concatenate([output1, output2, output3])
meta_model_1 = Dense(152, activation='relu',kernel_regularizer=l2(0.01), name='meta_model_1')(merged)
meta_model_2 = Dropout(0.5, name='meta_model_2')(meta_model_1)
meta_model_3 = Dense(64, activation='relu',kernel_regularizer=l2(0.01), name='meta_model_3')(meta_model_2)
meta_model_4 = Dropout(0.3, name='meta_model_4')(meta_model_3)
final_output = Dense(4, activation='softmax', name='final_output')(meta_model_4)

# pass the concatenated output to the meta-model
#final_output = meta_model(merged)

# create the final model with inputs from the individual models
# and output from the meta-model
final_model_1 = Model(inputs=[model_1.input,model_4.input,model_5.input], outputs=final_output)

# compile the final model
final_model_1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
#[2,3,4]
from keras.models import Model
from keras.layers import concatenate, Dense, Dropout

# assume you have three models named model1, model2, model3
# and a meta-model named dense_meta_model
model_2.trainable = False
model_3.trainable=False
model_4.trainable = False
# get the output layers of individual models
output1 = model_2.output
output2 = model_3.output
output3 = model_4.output

# concatenate the outputs of the individual models
merged = concatenate([output1, output2, output3])
meta_model_1 = Dense(152, activation='relu',kernel_regularizer=l2(0.01), name='meta_model_1')(merged)
meta_model_2 = Dropout(0.5, name='meta_model_2')(meta_model_1)
meta_model_3 = Dense(64, activation='relu',kernel_regularizer=l2(0.01), name='meta_model_3')(meta_model_2)
meta_model_4 = Dropout(0.3, name='meta_model_4')(meta_model_3)
final_output = Dense(4, activation='softmax', name='final_output')(meta_model_4)

# pass the concatenated output to the meta-model
#final_output = meta_model(merged)

# create the final model with inputs from the individual models
# and output from the meta-model
final_model_2 = Model(inputs=[model_2.input,model_3.input,model_4.input], outputs=final_output)

# compile the final model
final_model_2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
#[2,3,5]
from keras.models import Model
from keras.layers import concatenate, Dense, Dropout

# assume you have three models named model1, model2, model3
# and a meta-model named dense_meta_model
model_2.trainable = False
model_3.trainable=False
model_5.trainable = False
# get the output layers of individual models
output1 = model_2.output
output2 = model_3.output
output3 = model_5.output

# concatenate the outputs of the individual models
merged = concatenate([output1, output2, output3])
meta_model_1 = Dense(152, activation='relu',kernel_regularizer=l2(0.01), name='meta_model_1')(merged)
meta_model_2 = Dropout(0.5, name='meta_model_2')(meta_model_1)
meta_model_3 = Dense(64, activation='relu',kernel_regularizer=l2(0.01), name='meta_model_3')(meta_model_2)
meta_model_4 = Dropout(0.3, name='meta_model_4')(meta_model_3)
final_output = Dense(4, activation='softmax', name='final_output')(meta_model_4)

# pass the concatenated output to the meta-model
#final_output = meta_model(merged)

# create the final model with inputs from the individual models
# and output from the meta-model
final_model_3 = Model(inputs=[model_2.input,model_3.input,model_5.input], outputs=final_output)

# compile the final model
final_model_3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Prepare the input data for prediction
X_test = [x_test, x_test,x_test]
X_train=[x_train,x_train,x_train]
X_val=[x_val,x_val,x_val]

In [ ]:
final_model_1.fit(X_train, y_train, epochs=20,validation_data=(X_val,y_val))
final_model_2.fit(X_train, y_train, epochs=20,validation_data=(X_val,y_val))
final_model_3.fit(X_train, y_train, epochs=20,validation_data=(X_val,y_val))

In [ ]:
train_loss_1, train_acc_1 = final_model_1.evaluate(X_train,y_train)
val_loss_1, test_acc_1 = final_model_1.evaluate(X_test,y_test)
train_loss_1, val_acc_1 = final_model_1.evaluate(X_val,y_val)

print(f"Final training accuracy: {train_acc_1:.4f},Final validation accuracy: {val_acc_1:.4f},Final test accuracy: {test_acc_1:.4f}")
train_loss_2, train_acc_2 = final_model_2.evaluate(X_train,y_train)
val_loss_2, test_acc_2 = final_model_2.evaluate(X_test,y_test)
train_loss_2, val_acc_2 = final_model_2.evaluate(X_val,y_val)

print(f"Final training accuracy: {train_acc_2:.4f},Final validation accuracy: {val_acc_2:.4f},Final test accuracy: {test_acc_2:.4f}")
train_loss_3, train_acc_3 = final_model_3.evaluate(X_train,y_train)
val_loss_3, test_acc_3 = final_model_3.evaluate(X_test,y_test)
train_loss_3, val_acc_3 = final_model_3.evaluate(X_val,y_val)

print(f"Final training accuracy: {train_acc_1:.4f},Final validation accuracy: {val_acc_1:.4f},Final test accuracy: {test_acc_1:.4f}")

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import numpy as np

# convert one-hot encoded labels to integers
y_train_classes = np.argmax(y_train, axis=1)
y_val_classes = np.argmax(y_val, axis=1)

combination_accuracies = []

# loop over all base model combinations
for combination, predictions in base_model_predictions.items():
    # concatenate base model predictions
    train_predictions = np.concatenate([base_model.predict(x_train) for base_model in combination], axis=1)
    val_predictions = np.concatenate([base_model.predict(x_val) for base_model in combination], axis=1)

    # define the SVM meta-model
    svm = SVC(probability=True)
    # define hyperparameters to search over
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
    # perform grid search over hyperparameters
    svm_grid = GridSearchCV(svm, param_grid=param_grid, cv=5, n_jobs=-1)
    svm_grid.fit(train_predictions, y_train_classes)

    # get the predictions of the current base model combination on the validation set
    val_predictions = np.concatenate([base_model.predict(x_val) for base_model in combination], axis=1)

    # evaluate the SVM meta-model on the validation set
    val_accuracy = svm_grid.score(val_predictions, y_val_classes)
    train_accuracy = svm_grid.score(train_predictions, y_train_classes)

    # print the accuracy of the current base model combination
    print(f'Accuracy for {len(combination)} base models: train: {train_accuracy}, val: {val_accuracy}')

    # add the accuracy of the current base model combination to the list
    combination_accuracies.append((combination, train_accuracy, val_accuracy))



In [ ]:
sorted(combination_accuracies,key=lambda x: (x[2], x[1],5-len(x[0])),reverse=True)[0:5]

In [ ]:
#[model_2,model_3]
#[model_2,model_4]
#[model_3,model_4]

In [ ]:
from keras.models import Model
from keras.layers import concatenate, Dense, Dropout
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# assume you have three models named model1, model2, model3
# and a meta-model named svm_meta_model
model_2.trainable = False
model_3.trainable = False

# get predictions of model_2 and model_3 on the validation set
predictions_2_val = model_2.predict(x_val)
predictions_3_val = model_3.predict(x_val)

# concatenate the predictions into a single feature matrix
base_model_predictions = np.concatenate((predictions_2_val, predictions_3_val), axis=1)

# define the parameter grid for hyperparameter tuning
param_grid = {'C': [0.1, 1, 10, 100], 'gamma': [0.1, 1, 10, 100]}

# create the SVM meta model
svm_meta_model = SVC(kernel='rbf', probability=True)
y_val=[np.argmax(x) for x in y_val]
# perform hyperparameter tuning using grid search with cross-validation
svm_meta_model_tuned = GridSearchCV(svm_meta_model, param_grid, cv=5, scoring='accuracy')
svm_meta_model_tuned.fit(base_model_predictions, y_val)

# get the best hyperparameters
best_C = svm_meta_model_tuned.best_params_['C']
best_gamma = svm_meta_model_tuned.best_params_['gamma']

# create the final SVM meta model using the best hyperparameters
svm_meta_model_final = SVC(kernel='rbf', probability=True, C=best_C, gamma=best_gamma)

# get predictions of model_2 and model_3 on the test set
predictions_2_test = model_2.predict(x_test)
predictions_3_test = model_3.predict(x_test)

# concatenate the predictions into a single feature matrix
base_model_predictions_test = np.concatenate((predictions_2_test, predictions_3_test), axis=1)

# train the final SVM meta model on the concatenated base model predictions
svm_meta_model_final.fit(base_model_predictions, y_val)
y_test=[np.argmax(x) for x in y_test]
# evaluate the final SVM meta model on the test set
svm_meta_model_accuracy = svm_meta_model_final.score(base_model_predictions_test, y_test)

print('Final SVM Meta Model Accuracy:', svm_meta_model_accuracy)



In [ ]:
from keras.models import Model
from keras.layers import concatenate, Dense, Dropout
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# assume you have three models named model1, model2, model3
# and a meta-model named svm_meta_model
model_2.trainable = False
model_4.trainable = False

# get predictions of model_2 and model_3 on the validation set
predictions_2_val = model_2.predict(x_val)
predictions_3_val = model_4.predict(x_val)

# concatenate the predictions into a single feature matrix
base_model_predictions = np.concatenate((predictions_2_val, predictions_3_val), axis=1)

# define the parameter grid for hyperparameter tuning
param_grid = {'C': [0.1, 1, 10, 100], 'gamma': [0.1, 1, 10, 100]}

# create the SVM meta model
svm_meta_model = SVC(kernel='rbf', probability=True)
#y_val=[np.argmax(x) for x in y_val]
# perform hyperparameter tuning using grid search with cross-validation
svm_meta_model_tuned = GridSearchCV(svm_meta_model, param_grid, cv=5, scoring='accuracy')
svm_meta_model_tuned.fit(base_model_predictions, y_val)

# get the best hyperparameters
best_C = svm_meta_model_tuned.best_params_['C']
best_gamma = svm_meta_model_tuned.best_params_['gamma']

# create the final SVM meta model using the best hyperparameters
svm_meta_model_final = SVC(kernel='rbf', probability=True, C=best_C, gamma=best_gamma)

# get predictions of model_2 and model_3 on the test set
predictions_2_test = model_2.predict(x_test)
predictions_3_test = model_4.predict(x_test)

# concatenate the predictions into a single feature matrix
base_model_predictions_test = np.concatenate((predictions_2_test, predictions_3_test), axis=1)

# train the final SVM meta model on the concatenated base model predictions
svm_meta_model_final.fit(base_model_predictions, y_val)
#y_test=[np.argmax(x) for x in y_test]
# evaluate the final SVM meta model on the test set
svm_meta_model_accuracy = svm_meta_model_final.score(base_model_predictions_test, y_test)

print('Final SVM Meta Model Accuracy:', svm_meta_model_accuracy)


In [ ]:
from keras.models import Model
from keras.layers import concatenate, Dense, Dropout
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# assume you have three models named model1, model2, model3
# and a meta-model named svm_meta_model
model_3.trainable = False
model_4.trainable = False

# get predictions of model_2 and model_3 on the validation set
predictions_2_val = model_3.predict(x_val)
predictions_3_val = model_4.predict(x_val)

# concatenate the predictions into a single feature matrix
base_model_predictions = np.concatenate((predictions_2_val, predictions_3_val), axis=1)

# define the parameter grid for hyperparameter tuning
param_grid = {'C': [0.1, 1, 10, 100], 'gamma': [0.1, 1, 10, 100]}

# create the SVM meta model
svm_meta_model = SVC(kernel='rbf', probability=True)
#y_val=[np.argmax(x) for x in y_val]
# perform hyperparameter tuning using grid search with cross-validation
svm_meta_model_tuned = GridSearchCV(svm_meta_model, param_grid, cv=5, scoring='accuracy')
svm_meta_model_tuned.fit(base_model_predictions, y_val)

# get the best hyperparameters
best_C = svm_meta_model_tuned.best_params_['C']
best_gamma = svm_meta_model_tuned.best_params_['gamma']

# create the final SVM meta model using the best hyperparameters
svm_meta_model_final = SVC(kernel='rbf', probability=True, C=best_C, gamma=best_gamma)

# get predictions of model_2 and model_3 on the test set
predictions_2_test = model_3.predict(x_test)
predictions_3_test = model_4.predict(x_test)
predictions_2_train = model_3.predict(x_train)
predictions_3_train = model_4.predict(x_train)
predictions_2_val = model_3.predict(x_val)
predictions_3_val = model_4.predict(x_val)
# concatenate the predictions into a single feature matrix
base_model_predictions_test = np.concatenate((predictions_2_test, predictions_3_test), axis=1)
base_model_predictions_train = np.concatenate((predictions_2_train, predictions_3_train), axis=1)
base_model_predictions_val = np.concatenate((predictions_2_val, predictions_3_val), axis=1)

# train the final SVM meta model on the concatenated base model predictions
svm_meta_model_final.fit(base_model_predictions, y_val)

y_train=[np.argmax(x) for x in y_train]
# evaluate the final SVM meta model on the test set
svm_meta_model_accuracy = svm_meta_model_final.score(base_model_predictions_test, y_test)
svm_meta_model_accuracy_train = svm_meta_model_final.score(base_model_predictions_train, y_train)
svm_meta_model_accuracy_val = svm_meta_model_final.score(base_model_predictions_val, y_val)

print('Final SVM Meta Model Accuracy:', svm_meta_model_accuracy)
print('Final SVM Meta Model Accuracy_train:', svm_meta_model_accuracy_train)
print('Final SVM Meta Model Accuracy_val:', svm_meta_model_accuracy_val)
